In [3]:
# Install necessary libraries if not already installed
!pip install groq together openai

# Import necessary libraries
from together import Together
from openai import OpenAI

# Initialize Together and OpenAI clients with your API keys
together_client = Together(api_key="8a7afb786e80870e79e094bb105c9fd6f9ace6acff04c80fcf4f71d18e1aecf4")
openai_client = OpenAI(api_key="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", base_url="https://api.perplexity.ai")

# Define a system prompt to set the context for the model
system_prompt = """
You are an AI assistant that manages coffee orders in a virtual kiosk.
Your job is to understand customer requests in Korean, provide a natural language response confirming the order details, and output the order in JSON format.
The JSON should be structured for backend processing and include the following fields:
- "action" (string): either "create_order", "add_item", "modify_order", "cancel_order", "confirm_order" or "show_order_summary"
- "order_items" (list): each item should have details such as "drink", "size", "temperature", "quantity", "add_ons", and "extra_shots".

Respond with:
1. A conversational response in Korean confirming the order.
2. A structured JSON object for the backend.

Continue accepting requests until the customer confirms the order. Be consistent in understanding quantities, drink types, sizes, temperatures, and modifications.
"""

# Initialize an empty order list and an action loop
order_history = []
order_confirmed = False

# Define a function to get responses from the Together API
def get_response(prompt):
    response = together_client.chat.completions.create(
        model="meta-llama/Meta-Llama-3.1-405B-Instruct-Turbo",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ],
        max_tokens=500,
        temperature=0.1,
        top_p=1,
        top_k=50,
        repetition_penalty=1,
        stop=["<|eot_id|>"]
    )
    return response.choices[0].message.content

# Start the continuous order loop
print("Welcome to the virtual coffee kiosk! What would you like to order?")

while not order_confirmed:
    # Take user input
    customer_input = input("Customer: ")

    # Check if the customer confirms the order
    if "확인" in customer_input or "주문 완료" in customer_input:
        order_confirmed = True
        prompt = f"Customer's Input: {customer_input}\nConfirm the order and end the conversation."
    else:
        # Add context from previous orders
        if order_history:
            order_summary = "\n".join([f"{idx+1}. {item['drink']} {item['quantity']}잔 ({item['temperature']}, {item['size']})"
                                       for idx, item in enumerate(order_history)])
            prompt = f"Current Order:\n{order_summary}\n\nCustomer's New Input: \"{customer_input}\""
        else:
            prompt = f"Customer's New Input: \"{customer_input}\""

    # Get model response
    response = get_response(prompt)

    # Process response for JSON structure
    try:
        # Split response to capture JSON separately if needed
        natural_language_response, json_output = response.split('JSON Output:', 1)

        # Print the natural language response
        print("Kiosk:", natural_language_response.strip())

        # Evaluate JSON to Python dict
        json_data = eval(json_output.strip())

        # Add to order history if action is create_order or add_item
        if json_data["action"] in ["create_order", "add_item"]:
            order_history.extend(json_data["order_items"])

    except Exception as e:
        print("Error parsing the response:", e)
        print("Raw Response:", response)

# Final order confirmation message
print("\nThank you for your order! Here is the confirmed order:")
for idx, item in enumerate(order_history, 1):
    print(f"{idx}. {item['drink']} {item['quantity']}잔 ({item['temperature']}, {item['size']})")


Welcome to the virtual coffee kiosk! What would you like to order?
Customer: 아이스 라떼 두잔 주세요
Kiosk: "네, 아이스 라떼 두잔 주문하셨습니다. 추가 주문이나 변경 사항이 있으신가요?"
Error parsing the response: invalid syntax (<string>, line 1)
Raw Response: "네, 아이스 라떼 두잔 주문하셨습니다. 추가 주문이나 변경 사항이 있으신가요?"

JSON Output:
```
{
  "action": "create_order",
  "order_items": [
    {
      "drink": "latte",
      "size": "regular",
      "temperature": "ice",
      "quantity": 2,
      "add_ons": [],
      "extra_shots": 0
    }
  ]
}
```
Customer: 초콜릿라떼 두잔이랑 바닐라라떼 한잔 주세요
Error parsing the response: not enough values to unpack (expected 2, got 1)
Raw Response: "초콜릿라떼 두잔과 바닐라라떼 한잔을 주문하셨습니다. 맞으신가요?"

```json
{
  "action": "create_order",
  "order_items": [
    {
      "drink": "초콜릿라떼",
      "size": "regular",
      "temperature": "hot",
      "quantity": 2,
      "add_ons": [],
      "extra_shots": 0
    },
    {
      "drink": "바닐라라떼",
      "size": "regular",
      "temperature": "hot",
      "quantity": 1,
      "add_ons": [],
   

KeyboardInterrupt: Interrupted by user

In [10]:
# Import necessary libraries
from together import Together
from openai import OpenAI

# Initialize Together and OpenAI clients with your API keys
together_client = Together(api_key="8a7afb786e80870e79e094bb105c9fd6f9ace6acff04c80fcf4f71d18e1aecf4")
openai_client = OpenAI(api_key="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", base_url="https://api.perplexity.ai")

# Initialize conversation state variables
cumulative_order_summary = []  # Stores each item as formatted text for natural language response
order_history_json = []        # Stores all items in JSON format for backend processing
order_confirmed = False         # Tracks if the order is finalized

# Function to generate prompt based on cumulative history and new input
def generate_prompt(cumulative_order_summary, user_input):
    # Format the cumulative order summary as a single string
    formatted_history = "\n".join(cumulative_order_summary) if cumulative_order_summary else "none"
    base_prompt = f"""
    You are operating a virtual coffee kiosk that receives speech-to-text (STT) inputs from customers placing coffee orders. Your role is to understand and process these inputs, respond naturally in Korean, and generate a structured JSON file with the correct details for backend processing.

    **Key Requirements**:
    - **Menu Items**: The kiosk offers the following drinks:
        - Hot and Iced Coffee: 아메리카노, 라떼, 카푸치노, 카페모카, 바닐라라떼, 에스프레소, 카라멜마끼아또
        - Tea: 허브티, 홍차
        - Specialty Drinks: 초콜릿라떼, 레몬에이드, 복숭아아이스티, 딸기스무디, 망고스무디, 키위주스, 토마토주스
    - **Default Values**:
        - Use default size "미디움" and temperature "핫" only if the customer does not specify these details.
    - **Do Not Make Assumptions**:
        - If the customer specifies temperature or size, do not override it with defaults. For instance, if they say "아이스 라떼 두잔 주세요", the output should indicate "아이스" without changing it to "핫".

    **Customer Input and Expected Output Format**:
    - Each response should have:
      1. **Natural Language Confirmation**: Respond in Korean, starting with an action confirmation such as "[Drink] [quantity] 주문되었습니다." and follow with a full summary of all items ordered so far, beginning with "지금까지 주문하신 내용은 다음과 같습니다:".(also if there are any instance in history it should be added after this sentence)
      2. **Structured JSON Output**: Each JSON output should only contain the items directly requested in the latest input, not a full history.

    **JSON Output Format**:
    - The JSON should be structured as follows:
      ```json
      {{
          "action": "[action_type]",
          "order_items": [
              {{
                  "drink": "[Drink Name]",
                  "size": "[Size]",
                  "temperature": "[Temperature]",
                  "quantity": [Quantity],
                  "add_ons": [List of add-ons if any],
                  "extra_shots": [Number of extra shots if any]
              }}
          ]
      }}
      ```
      - **Example JSON Output**:
        ```json
        {{
            "action": "create_order",
            "order_items": [
                {{
                    "drink": "아메리카노",
                    "size": "미디움",
                    "temperature": "핫",
                    "quantity": 1,
                    "add_ons": [],
                    "extra_shots": 0
                }}
            ]
        }}
        ```

    **Available Actions for JSON Output**:
    - **create_order**: For new drink orders.
    - **add_item**: For adding a new item to the current order.
    - **modify_order**: For changing an existing item (e.g., modifying size or temperature).
    - **cancel_order**: To remove an order item or reset the order.
    - **recommend_closest_item**: If a requested item is unavailable, recommend the closest item.
    - **show_order_summary**: Display a summary of all items ordered so far.
    - **complete_order**: Finalize the order after confirmation.

    **Specific Scenarios and Expected Outputs**:
    - **Creating a New Order**:
      - **Customer Input**: "아메리카노 4잔 주세요."
      - **Natural Language Response**: "아메리카노 4잔 주문되었습니다. 지금까지 주문하신 내용은 다음과 같습니다:
      -아메리카노 4잔 (핫, 미디움)"
      - **JSON Output**:
        ```json
        {{
          "action": "create_order",
          "order_items": [
            {{
              "drink": "아메리카노",
              "size": "미디움",
              "temperature": "핫",
              "quantity": 4,
              "add_ons": [],
              "extra_shots": 0
            }}
          ]
        }}
        ```

    - **Requesting Order Summary**:
      - **Customer Input**: "내가 지금까지 뭘 주문했지?"
      - **Natural Language Response**: "지금까지 주문하신 내용은 다음과 같습니다:
      -아메리카노 4잔(핫, 미디움)
      -카페라떼 라지 2잔 (핫, 라지)"
      - **JSON Output**: None (as it is just a summary request without any new action).

    - **Modifying an Existing Order**:
      - **Customer Input**: "주문한거 아이스 라떼로 바꿔줘."
      - **Natural Language Response**: "주문이 아메리카노에서 아이스 라떼로 변경되었습니다. 주문하신 내용은 다음과 같습니다:
      -라떼 1잔 (아이스, 미디움)"
      - **JSON Output**:
        ```json
        {{
          "action": "modify_order",
          "old_drink": "아메리카노",
          "new_drink": "라떼",
          "size": "미디움",
          "temperature": "아이스",
          "quantity": 1,
          "add_ons": [],
          "extra_shots": 0
        }}
        ```

    - **Short Names or Misspellings**:
      - Recognize common shorthand or misspellings. For example:
        - "아아" should be interpreted as "아이스 아메리카노".
        - "뜨아" should be interpreted as "핫 아메리카노".

    - **Unavailable Items**:
      - If the customer requests an item not on the menu, respond politely and recommend a similar item if available.
      - **Example**:
        - **Customer Input**: "초코라떼 주세요."
        - **Natural Language Response**: "죄송합니다, 초코라떼는 메뉴에 없습니다. 대신 초콜릿라떼를 추천드립니다."
        - **JSON Output**:
          ```json
          {{
            "action": "recommend_closest_item",
            "requested_item": "초코라떼",
            "recommended_item": "초콜릿라떼"
          }}
          ```

    - **Order Confirmation**:
      - **Customer Input**: "주문 완료할게요."
      - **Natural Language Response**: "주문이 완료되었습니다. 결제는 카드리더기를 사용해주세요. 감사합니다."
      - **JSON Output**: Clear/reset for the next session.

    **Response Rules**:
    - Treat each new input as part of the same order until "주문 완료할게" is received, which finalizes the order.
    - Always confirm the latest action first in the natural language response, followed by a full order summary.
    - Ensure each JSON output reflects only the latest request, not the entire order history.

    Based on this information, generate the appropriate natural language response and JSON output if **Current Conversation History**:
    {conversation_history}

    and **Customer's New Input**: "{user_input}".
    """

    return base_prompt.strip()

# Function to get response from Together API
def get_response(prompt):
    response = together_client.chat.completions.create(
        model="meta-llama/Meta-Llama-3.1-405B-Instruct-Turbo",
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": prompt}
        ],
        max_tokens=500,
        temperature=0.1,
        top_p=1,
        top_k=50,
        repetition_penalty=1,
        stop=["<|eot_id|>"]
    )
    return response.choices[0].message.content

# Function to parse response into natural language and JSON output
def parse_response(response):
    try:
        natural_language_response, json_output = response.split("JSON Output:")
        json_data = eval(json_output.strip())
        return natural_language_response.strip(), json_data
    except ValueError as e:
        print("Error parsing response:", e)
        return response, None

# Main interaction loop
print("Welcome to the virtual coffee kiosk! What would you like to order?")

while not order_confirmed:
    # Take user input
    user_input = input("Customer: ")

    # Check if customer confirms the order
    if "주문 완료할게" in user_input:
        order_confirmed = True
        print("Kiosk: 주문이 완료되었습니다. 결제는 카드리더기를 사용해주세요. 감사합니다.")
        cumulative_order_summary.clear()
        order_history_json.clear()
        continue

    # Generate the prompt based on cumulative history and new user input
    prompt = generate_prompt(cumulative_order_summary, user_input)

    # Get the model's response
    response = get_response(prompt)

    # Parse the model's response
    natural_language_response, json_output = parse_response(response)

    # Print the natural language response for the customer
    print("Kiosk:", natural_language_response)

    # Process JSON output if available
    if json_output:
        # Save only the latest item in the formatted summary for cumulative order summary
        for item in json_output.get("order_items", []):
            item_summary = f"{item['drink']} {item['quantity']}잔 ({item['temperature']}, {item['size']})"
            cumulative_order_summary.append(item_summary)  # Append to cumulative text summary
            order_history_json.append(item)               # Append to cumulative JSON order

    # Special Handling for Order Summary Request
    if "내가 지금까지 뭘 주문했지" in user_input:
        # Display cumulative order summary without JSON for order history requests
        print("Kiosk:", "지금까지의 주문내역입니다:\n" + "\n".join(cumulative_order_summary))

    # Handle order cancellation requests
    if "주문 취소" in user_input:
        # Clear entire order for a reset, or selectively based on more detailed criteria if needed
        print("Kiosk: 주문이 취소되었습니다. 새 주문을 시작해 주세요.")
        cumulative_order_summary.clear()
        order_history_json.clear()

# End of the session
print("Thank you for using the coffee kiosk!")



Welcome to the virtual coffee kiosk! What would you like to order?
Customer: 아메리카노 4잔 주세요
Error parsing response: not enough values to unpack (expected 2, got 1)
Kiosk: **Natural Language Response**:
아메리카노 4잔 주문되었습니다. 지금까지 주문하신 내용은 다음과 같습니다:
- 아메리카노 4잔 (핫, 미디움)

**JSON Output**:
```json
{
  "action": "create_order",
  "order_items": [
    {
      "drink": "아메리카노",
      "size": "미디움",
      "temperature": "핫",
      "quantity": 4,
      "add_ons": [],
      "extra_shots": 0
    }
  ]
}
```
Customer: 에스프레소 두잔 시원하게 주실래요?
Error parsing response: not enough values to unpack (expected 2, got 1)
Kiosk: **Natural Language Response**:
에스프레소 2잔 주문되었습니다. 주문하신 에스프레소는 시원하게(아이스) 준비하겠습니다. 지금까지 주문하신 내용은 다음과 같습니다:
- 에스프레소 2잔 (아이스, 미디움)

**JSON Output**:
```json
{
  "action": "create_order",
  "order_items": [
    {
      "drink": "에스프레소",
      "size": "미디움",
      "temperature": "아이스",
      "quantity": 2,
      "add_ons": [],
      "extra_shots": 0
    }
  ]
}
```

In this response, I recognized the

KeyboardInterrupt: Interrupted by user

In [18]:
# Import necessary libraries
from together import Together
from openai import OpenAI

# Initialize Together and OpenAI clients with your API keys
together_client = Together(api_key="8a7afb786e80870e79e094bb105c9fd6f9ace6acff04c80fcf4f71d18e1aecf4")
openai_client = OpenAI(api_key="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", base_url="https://api.perplexity.ai")

# Initialize conversation state variables
cumulative_order_summary = []  # Stores each item as formatted text for natural language response
order_history_json = []        # Stores all items in JSON format for backend processing
order_confirmed = False         # Tracks if the order is finalized

# Function to generate prompt based on cumulative history and new input
def generate_prompt(cumulative_order_summary, user_input):
    # Format the cumulative order summary as a single string
    formatted_history = "\n".join(cumulative_order_summary) if cumulative_order_summary else "none"
    base_prompt = f"""
    You are operating a virtual coffee kiosk that receives speech-to-text (STT) inputs from customers placing coffee orders. Your role is to understand and process these inputs, respond naturally in Korean, and generate a structured JSON file with the correct details for backend processing.

    **Key Requirements**:
    - **Menu Items**: The kiosk offers the following drinks:
        - Hot and Iced Coffee: 아메리카노, 라떼, 카푸치노, 카페모카, 바닐라라떼, 에스프레소, 카라멜마끼아또
        - Tea: 허브티, 홍차
        - Specialty Drinks: 초콜릿라떼, 레몬에이드, 복숭아아이스티, 딸기스무디, 망고스무디, 키위주스, 토마토주스
    - **Default Values**:
        - Use default size "미디움" and temperature "핫" only if the customer does not specify these details.
    - **Do Not Make Assumptions**:
        - If the customer specifies temperature or size, do not override it with defaults. For instance, if they say "아이스 라떼 두잔 주세요", the output should indicate "아이스" without changing it to "핫".

    **Customer Input and Expected Output Format**:
    - Each response should have:
      1. **Natural Language Confirmation**: Respond in Korean, starting with an action confirmation such as "[Drink] [quantity] 주문되었습니다." and follow with a full summary of all items ordered so far, beginning with "지금까지 주문하신 내용은 다음과 같습니다:".(also if there are any instance in history it should be added after this sentence)
      2. **Structured JSON Output**: Each JSON output should only contain the items directly requested in the latest input, not a full history.

    **JSON Output Format**:
    - The JSON should be structured as follows:
      ```json
      {{
          "action": "[action_type]",
          "order_items": [
              {{
                  "drink": "[Drink Name]",
                  "size": "[Size]",
                  "temperature": "[Temperature]",
                  "quantity": [Quantity],
                  "add_ons": [List of add-ons if any],
                  "extra_shots": [Number of extra shots if any]
              }}
          ]
      }}
      ```
      - **Example JSON Output**:
        ```json
        {{
            "action": "create_order",
            "order_items": [
                {{
                    "drink": "아메리카노",
                    "size": "미디움",
                    "temperature": "핫",
                    "quantity": 1,
                    "add_ons": [],
                    "extra_shots": 0
                }}
            ]
        }}
        ```

    **Available Actions for JSON Output**:
    - **create_order**: For new drink orders.
    - **add_item**: For adding a new item to the current order.
    - **modify_order**: For changing an existing item (e.g., modifying size or temperature).
    - **cancel_order**: To remove an order item or reset the order.
    - **recommend_closest_item**: If a requested item is unavailable, recommend the closest item.
    - **show_order_summary**: Display a summary of all items ordered so far.
    - **complete_order**: Finalize the order after confirmation.

    **Specific Scenarios and Expected Outputs**:
    - **Creating a New Order**:
      - **Customer Input**: "아메리카노 4잔 주세요."
      - **Natural Language Response**: "아메리카노 4잔 주문되었습니다. 지금까지 주문하신 내용은 다음과 같습니다:
      -아메리카노 4잔 (핫, 미디움)"
      - **JSON Output**:
        ```json
        {{
          "action": "create_order",
          "order_items": [
            {{
              "drink": "아메리카노",
              "size": "미디움",
              "temperature": "핫",
              "quantity": 4,
              "add_ons": [],
              "extra_shots": 0
            }}
          ]
        }}
        ```

    - **Requesting Order Summary**:
      - **Customer Input**: "내가 지금까지 뭘 주문했지?"
      - **Natural Language Response**: "지금까지 주문하신 내용은 다음과 같습니다:
      -아메리카노 4잔(핫, 미디움)
      -카페라떼 라지 2잔 (핫, 라지)"
      - **JSON Output**: None (as it is just a summary request without any new action).

    - **Modifying an Existing Order**:
      - **Customer Input**: "주문한거 아이스 라떼로 바꿔줘."
      - **Natural Language Response**: "주문이 아메리카노에서 아이스 라떼로 변경되었습니다. 주문하신 내용은 다음과 같습니다:
      -라떼 1잔 (아이스, 미디움)"
      - **JSON Output**:
        ```json
        {{
          "action": "modify_order",
          "old_drink": "아메리카노",
          "new_drink": "라떼",
          "size": "미디움",
          "temperature": "아이스",
          "quantity": 1,
          "add_ons": [],
          "extra_shots": 0
        }}
        ```

    - **Short Names or Misspellings**:
      - Recognize common shorthand or misspellings. For example:
        - "아아" should be interpreted as "아이스 아메리카노".
        - "뜨아" should be interpreted as "핫 아메리카노".

    - **Unavailable Items**:
      - If the customer requests an item not on the menu, respond politely and recommend a similar item if available.
      - **Example**:
        - **Customer Input**: "초코라떼 주세요."
        - **Natural Language Response**: "죄송합니다, 초코라떼는 메뉴에 없습니다. 대신 초콜릿라떼를 추천드립니다."
        - **JSON Output**:
          ```json
          {{
            "action": "recommend_closest_item",
            "requested_item": "초코라떼",
            "recommended_item": "초콜릿라떼"
          }}
          ```

    - **Order Confirmation**:
      - **Customer Input**: "주문 완료할게요."
      - **Natural Language Response**: "주문이 완료되었습니다. 결제는 카드리더기를 사용해주세요. 감사합니다."
      - **JSON Output**: Clear/reset for the next session.

    **Response Rules**:
    - Treat each new input as part of the same order until "주문 완료할게" is received, which finalizes the order.
    - Always confirm the latest action first in the natural language response, followed by a full order summary.
    - Ensure each JSON output reflects only the latest request, not the entire order history.

    Based on this information, generate the appropriate natural language response and JSON output
    if **Current Conversation History**:
    {formatted_history}

    and **Customer's New Input**: "{user_input}".
    """
    print("Formatted History for Current Prompt:", formatted_history)
    return base_prompt.strip()

# Function to get response from Together API
def get_response(prompt):
    response = together_client.chat.completions.create(
        model="meta-llama/Meta-Llama-3.1-405B-Instruct-Turbo",
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": prompt}
        ],
        max_tokens=500,
        temperature=0.1,
        top_p=1,
        top_k=50,
        repetition_penalty=1,
        stop=["<|eot_id|>"]
    )
    return response.choices[0].message.content

# Function to parse response into natural language and JSON output
def parse_response(response):
    try:
        natural_language_response, json_output = response.split("JSON Output:")
        json_data = eval(json_output.strip())
        return natural_language_response.strip(), json_data
    except ValueError as e:
        print("Error parsing response:", e)
        return response, None

# Main interaction loop
print("Welcome to the virtual coffee kiosk! What would you like to order?")

while not order_confirmed:
    # Take user input
    user_input = input("Customer: ")

    # Check if customer confirms the order
    if "주문 완료할게" in user_input:
        order_confirmed = True
        print("Kiosk: 주문이 완료되었습니다. 결제는 카드리더기를 사용해주세요. 감사합니다.")
        cumulative_order_summary.clear()
        order_history_json.clear()
        continue

    # Generate the prompt based on cumulative history and new user input
    prompt = generate_prompt(cumulative_order_summary, user_input)

    # Get the model's response
    response = get_response(prompt)

    # Parse the model's response
    natural_language_response, json_output = parse_response(response)

    # Generate the cumulative natural language response with full order summary
    if json_output:
        # Append the latest order item(s) to the cumulative summary
        for item in json_output.get("order_items", []):
            item_summary = f"{item['drink']} {item['quantity']}잔 ({item['temperature']}, {item['size']})"
            cumulative_order_summary.append(item_summary)  # Append to cumulative text summary
            order_history_json.append(item)               # Append to cumulative JSON order

        # Debug: Print cumulative order summary to verify correct updates
        print("Updated Cumulative Order Summary:", cumulative_order_summary)

        # Create the full natural language response by confirming the latest action
        # and then displaying the cumulative order summary.
        action_confirmation = natural_language_response.split("\n")[0]
        full_summary = "주문하신 내용은 다음과 같습니다:\n" + "\n".join(cumulative_order_summary)
        natural_language_response = f"{action_confirmation}\n{full_summary}"

    # Print the final natural language response for the customer
    print("Kiosk:", natural_language_response)

    # Special Handling for Order Summary Request
    if "내가 지금까지 뭘 주문했지" in user_input:
        # Display cumulative order summary without JSON for order history requests
        print("Kiosk:", "지금까지의 주문내역입니다:\n" + "\n".join(cumulative_order_summary))

    # Handle order cancellation requests
    if "주문 취소" in user_input:
        # Clear entire order for a reset, or selectively based on more detailed criteria if needed
        print("Kiosk: 주문이 취소되었습니다. 새 주문을 시작해 주세요.")
        cumulative_order_summary.clear()
        order_history_json.clear()

# End of the session
print("Thank you for using the coffee kiosk!")


Welcome to the virtual coffee kiosk! What would you like to order?
Customer: 아메리카노 한잔 주새요
Formatted History for Current Prompt: none
Current Item List for Next Prompt: 
Error parsing response - possibly incorrect format: not enough values to unpack (expected 2, got 1)
Full Response: **Natural Language Response**:
아메리카노 한잔 주문되었습니다. 지금까지 주문하신 내용은 다음과 같습니다:
- 아메리카노 1잔 (핫, 미디움)

**JSON Output**:
```json
{
  "action": "create_order",
  "order_items": [
    {
      "drink": "아메리카노",
      "size": "미디움",
      "temperature": "핫",
      "quantity": 1,
      "add_ons": [],
      "extra_shots": 0
    }
  ]
}
```

Note: Since the customer did not specify the size or temperature, the default values of "미디움" and "핫" are used.
Kiosk: **Natural Language Response**:
아메리카노 한잔 주문되었습니다. 지금까지 주문하신 내용은 다음과 같습니다:
- 아메리카노 1잔 (핫, 미디움)

**JSON Output**:
```json
{
  "action": "create_order",
  "order_items": [
    {
      "drink": "아메리카노",
      "size": "미디움",
      "temperature": "핫",
      "quantity": 1,
    

KeyboardInterrupt: Interrupted by user

In [19]:
# Import necessary libraries
from together import Together
from openai import OpenAI

# Initialize Together and OpenAI clients with your API keys
together_client = Together(api_key="8a7afb786e80870e79e094bb105c9fd6f9ace6acff04c80fcf4f71d18e1aecf4")
openai_client = OpenAI(api_key="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", base_url="https://api.perplexity.ai")

# Initialize conversation state variables
cumulative_order_summary = []  # Stores each item as formatted text for natural language response
order_history_json = []        # Stores all items in JSON format for backend processing
order_confirmed = False         # Tracks if the order is finalized

# Function to generate prompt based on cumulative history and new input
def generate_prompt(cumulative_order_summary, user_input):
    # Format the cumulative order summary as a single string
    formatted_history = "\n".join(cumulative_order_summary) if cumulative_order_summary else "none"
    base_prompt = f"""
    You are operating a virtual coffee kiosk that receives speech-to-text (STT) inputs from customers placing coffee orders. Your role is to understand and process these inputs, respond naturally in Korean, and generate a structured JSON file with the correct details for backend processing.

    **Key Requirements**:
    - **Menu Items**: The kiosk offers the following drinks:
        - Hot and Iced Coffee: 아메리카노, 라떼, 카푸치노, 카페모카, 바닐라라떼, 에스프레소, 카라멜마끼아또
        - Tea: 허브티, 홍차
        - Specialty Drinks: 초콜릿라떼, 레몬에이드, 복숭아아이스티, 딸기스무디, 망고스무디, 키위주스, 토마토주스
    - **Default Values**:
        - Use default size "미디움" and temperature "핫" only if the customer does not specify these details.
    - **Do Not Make Assumptions**:
        - If the customer specifies temperature or size, do not override it with defaults. For instance, if they say "아이스 라떼 두잔 주세요", the output should indicate "아이스" without changing it to "핫".

    **Customer Input and Expected Output Format**:
    - Each response should have:
      1. **Natural Language Confirmation**: Respond in Korean, starting with an action confirmation such as "[Drink] [quantity] 주문되었습니다." and follow with a full summary of all items ordered so far, beginning with "지금까지 주문하신 내용은 다음과 같습니다:".(also if there are any instance in history it should be added after this sentence)
      2. **Structured JSON Output**: Each JSON output should only contain the items directly requested in the latest input, not a full history.

    **JSON Output Format**:
    - The JSON should be structured as follows:
      ```json
      {{
          "action": "[action_type]",
          "order_items": [
              {{
                  "drink": "[Drink Name]",
                  "size": "[Size]",
                  "temperature": "[Temperature]",
                  "quantity": [Quantity],
                  "add_ons": [List of add-ons if any],
                  "extra_shots": [Number of extra shots if any]
              }}
          ]
      }}
      ```
      - **Example JSON Output**:
        ```json
        {{
            "action": "create_order",
            "order_items": [
                {{
                    "drink": "아메리카노",
                    "size": "미디움",
                    "temperature": "핫",
                    "quantity": 1,
                    "add_ons": [],
                    "extra_shots": 0
                }}
            ]
        }}
        ```

    **Available Actions for JSON Output**:
    - **create_order**: For new drink orders.
    - **add_item**: For adding a new item to the current order.
    - **modify_order**: For changing an existing item (e.g., modifying size or temperature).
    - **cancel_order**: To remove an order item or reset the order.
    - **recommend_closest_item**: If a requested item is unavailable, recommend the closest item.
    - **show_order_summary**: Display a summary of all items ordered so far.
    - **complete_order**: Finalize the order after confirmation.

    **Specific Scenarios and Expected Outputs**:
    - **Creating a New Order**:
      - **Customer Input**: "아메리카노 4잔 주세요."
      - **Natural Language Response**: "아메리카노 4잔 주문되었습니다. 지금까지 주문하신 내용은 다음과 같습니다:
      -아메리카노 4잔 (핫, 미디움)"
      - **JSON Output**:
        ```json
        {{
          "action": "create_order",
          "order_items": [
            {{
              "drink": "아메리카노",
              "size": "미디움",
              "temperature": "핫",
              "quantity": 4,
              "add_ons": [],
              "extra_shots": 0
            }}
          ]
        }}
        ```

    - **Requesting Order Summary**:
      - **Customer Input**: "내가 지금까지 뭘 주문했지?"
      - **Natural Language Response**: "지금까지 주문하신 내용은 다음과 같습니다:
      -아메리카노 4잔(핫, 미디움)
      -카페라떼 라지 2잔 (핫, 라지)"
      - **JSON Output**: None (as it is just a summary request without any new action).

    - **Modifying an Existing Order**:
      - **Customer Input**: "주문한거 아이스 라떼로 바꿔줘."
      - **Natural Language Response**: "주문이 아메리카노에서 아이스 라떼로 변경되었습니다. 주문하신 내용은 다음과 같습니다:
      -라떼 1잔 (아이스, 미디움)"
      - **JSON Output**:
        ```json
        {{
          "action": "modify_order",
          "old_drink": "아메리카노",
          "new_drink": "라떼",
          "size": "미디움",
          "temperature": "아이스",
          "quantity": 1,
          "add_ons": [],
          "extra_shots": 0
        }}
        ```

    - **Short Names or Misspellings**:
      - Recognize common shorthand or misspellings. For example:
        - "아아" should be interpreted as "아이스 아메리카노".
        - "뜨아" should be interpreted as "핫 아메리카노".

    - **Unavailable Items**:
      - If the customer requests an item not on the menu, respond politely and recommend a similar item if available.
      - **Example**:
        - **Customer Input**: "초코라떼 주세요."
        - **Natural Language Response**: "죄송합니다, 초코라떼는 메뉴에 없습니다. 대신 초콜릿라떼를 추천드립니다."
        - **JSON Output**:
          ```json
          {{
            "action": "recommend_closest_item",
            "requested_item": "초코라떼",
            "recommended_item": "초콜릿라떼"
          }}
          ```

    - **Order Confirmation**:
      - **Customer Input**: "주문 완료할게요."
      - **Natural Language Response**: "주문이 완료되었습니다. 결제는 카드리더기를 사용해주세요. 감사합니다."
      - **JSON Output**: Clear/reset for the next session.

    **Response Rules**:
    - Treat each new input as part of the same order until "주문 완료할게" is received, which finalizes the order.
    - Always confirm the latest action first in the natural language response, followed by a full order summary.
    - Ensure each JSON output reflects only the latest request, not the entire order history.

    Based on this information, generate the appropriate natural language response and JSON output
    if **Current Conversation History**:
    {formatted_history}

    and **Customer's New Input**: "{user_input}".
    """

    print("Formatted History for Current Prompt:", formatted_history)  # Debugging: Print formatted history
    return base_prompt.strip()

# Function to get response from Together API
def get_response(prompt):
    response = together_client.chat.completions.create(
        model="meta-llama/Meta-Llama-3.1-405B-Instruct-Turbo",
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": prompt}
        ],
        max_tokens=500,
        temperature=0.1,
        top_p=1,
        top_k=50,
        repetition_penalty=1,
        stop=["<|eot_id|>"]
    )
    return response.choices[0].message.content

# Function to parse response into natural language and JSON output
def parse_response(response):
    try:
        natural_language_response, json_output = response.split("JSON Output:")
        json_data = json.loads(json_output.strip())
        return natural_language_response.strip(), json_data
    except ValueError as e:
        print("Error parsing response:", e)
        return response, None

# Main interaction loop
print("Welcome to the virtual coffee kiosk! What would you like to order?")

while not order_confirmed:
    # Take user input
    user_input = input("Customer: ")

    # Check if customer confirms the order
    if "주문 완료할게" in user_input:
        order_confirmed = True
        print("Kiosk: 주문이 완료되었습니다. 결제는 카드리더기를 사용해주세요. 감사합니다.")
        cumulative_order_summary.clear()
        order_history_json.clear()
        continue

    # Generate the prompt based on cumulative history and new user input
    prompt = generate_prompt(cumulative_order_summary, user_input)

    # Get the model's response
    response = get_response(prompt)

    # Parse the model's response
    natural_language_response, json_output = parse_response(response)

    # Generate the cumulative natural language response with full order summary
    if json_output:
        # Append the latest order item(s) to the cumulative summary
        for item in json_output.get("order_items", []):
            item_summary = f"{item['drink']} {item['quantity']}잔 ({item['temperature']}, {item['size']})"
            cumulative_order_summary.append(item_summary)  # Append to cumulative text summary
            order_history_json.append(item)               # Append to cumulative JSON order

        # Debug: Print cumulative order summary to verify correct updates
        print("Updated Cumulative Order Summary:", cumulative_order_summary)

        # Create the full natural language response by confirming the latest action
        # and then displaying the cumulative order summary.
        action_confirmation = natural_language_response.split("\n")[0]
        full_summary = "지금까지 주문하신 내용은 다음과 같습니다:\n" + "\n".join(cumulative_order_summary)
        natural_language_response = f"{action_confirmation}\n{full_summary}"

    # Print the final natural language response for the customer
    print("Kiosk:", natural_language_response)

    # Special Handling for Order Summary Request
    if "내가 지금까지 뭘 주문했지" in user_input:
        # Display cumulative order summary without JSON for order history requests
        print("Kiosk:", "지금까지의 주문내역입니다:\n" + "\n".join(cumulative_order_summary))

    # Handle order cancellation requests
    if "주문 취소" in user_input:
        # Clear entire order for a reset, or selectively based on more detailed criteria if needed
        print("Kiosk: 주문이 취소되었습니다. 새 주문을 시작해 주세요.")
        cumulative_order_summary.clear()
        order_history_json.clear()

# End of the session
print("Thank you for using the coffee kiosk!")

Welcome to the virtual coffee kiosk! What would you like to order?
Customer: 아메리카노 한잔 주세요
Formatted History for Current Prompt: none
Error parsing response: not enough values to unpack (expected 2, got 1)
Kiosk: **Natural Language Response**:
아메리카노 한잔 주문되었습니다. 지금까지 주문하신 내용은 다음과 같습니다:
- 아메리카노 1잔 (핫, 미디움)

**JSON Output**:
```json
{
  "action": "create_order",
  "order_items": [
    {
      "drink": "아메리카노",
      "size": "미디움",
      "temperature": "핫",
      "quantity": 1,
      "add_ons": [],
      "extra_shots": 0
    }
  ]
}
```
Customer: 아이스 라떼 두잔 주세요
Formatted History for Current Prompt: none
Error parsing response: not enough values to unpack (expected 2, got 1)
Kiosk: **Natural Language Response**:
"아이스 라떼 2잔 주문되었습니다. 지금까지 주문하신 내용은 다음과 같습니다: 
- 아이스 라떼 2잔 (아이스, 미디움)"

**JSON Output**:
```json
{
  "action": "create_order",
  "order_items": [
    {
      "drink": "라떼",
      "size": "미디움",
      "temperature": "아이스",
      "quantity": 2,
      "add_ons": [],
      "extra_shots": 0


KeyboardInterrupt: Interrupted by user

In [24]:
#best so far
# Import necessary libraries
from together import Together
from openai import OpenAI


# Initialize Together and OpenAI clients with your API keys
together_client = Together(api_key="8a7afb786e80870e79e094bb105c9fd6f9ace6acff04c80fcf4f71d18e1aecf4")
openai_client = OpenAI(api_key="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", base_url="https://api.perplexity.ai")

# Initialize conversation history and order confirmation flag
conversation_history = []  # Stores each customer input as a string entry in the format: "Customer's X Input: [input]"
order_confirmed = False     # Tracks if the order is finalized

# Function to generate prompt based on conversation history and new input
def generate_prompt(conversation_history, user_input):
    # Format the cumulative order summary as a single string
    formatted_history = "\n".join(conversation_history) if conversation_history else "none"
    base_prompt = f"""
    You are operating a virtual coffee kiosk that receives speech-to-text (STT) inputs from customers placing coffee orders. Your role is to understand and process these inputs, respond naturally in Korean, and generate a structured JSON file with the correct details for backend processing.

    **Key Requirements**:
    - **Menu Items**: The kiosk offers the following drinks:
        - Hot and Iced Coffee: 아메리카노, 라떼, 카푸치노, 카페모카, 바닐라라떼, 에스프레소, 카라멜마끼아또
        - Tea: 허브티, 홍차
        - Specialty Drinks: 초콜릿라떼, 레몬에이드, 복숭아아이스티, 딸기스무디, 망고스무디, 키위주스, 토마토주스
    - **Default Values**:
        - Use default size "미디움" and temperature "핫" only if the customer does not specify these details.
    - **Do Not Make Assumptions**:
        - If the customer specifies temperature or size, do not override it with defaults. For instance, if they say "아이스 라떼 두잔 주세요", the output should indicate "아이스" without changing it to "핫".

    **Customer Input and Expected Output Format**:
    - Each response should have:
      1. **Natural Language Confirmation**: Respond in Korean, starting with an action confirmation such as "[Drink] [quantity] 주문되었습니다." and follow with a full summary of all items ordered so far, beginning with "지금까지 주문하신 내용은 다음과 같습니다:".(also if there are any instance in history it should be added after this sentence)
      2. **Structured JSON Output**: Each JSON output should only contain the items directly requested in the latest input, not a full history.

    **JSON Output Format**:
    - The JSON should be structured as follows:
      ```json
      {{
          "action": "[action_type]",
          "order_items": [
              {{
                  "drink": "[Drink Name]",
                  "size": "[Size]",
                  "temperature": "[Temperature]",
                  "quantity": [Quantity],
                  "add_ons": [List of add-ons if any],
                  "extra_shots": [Number of extra shots if any]
              }}
          ]
      }}
      ```
      - **Example JSON Output**:
        ```json
        {{
            "action": "create_order",
            "order_items": [
                {{
                    "drink": "아메리카노",
                    "size": "미디움",
                    "temperature": "핫",
                    "quantity": 1,
                    "add_ons": [],
                    "extra_shots": 0
                }}
            ]
        }}
        ```

    **Available Actions for JSON Output**:
    - **create_order**: For new drink orders.
    - **add_item**: For adding a new item to the current order.
    - **modify_order**: For changing an existing item (e.g., modifying size or temperature).
    - **cancel_order**: To remove an order item or reset the order.
    - **recommend_closest_item**: If a requested item is unavailable, recommend the closest item.
    - **show_order_summary**: Display a summary of all items ordered so far.
    - **complete_order**: Finalize the order after confirmation.

    **Specific Scenarios and Expected Outputs**:
    - **Creating a New Order**:
      - **Customer Input**: "아메리카노 4잔 주세요."
      - **Natural Language Response**: "아메리카노 4잔 주문되었습니다. 지금까지 주문하신 내용은 다음과 같습니다:
      -아메리카노 4잔 (핫, 미디움)"
      - **JSON Output**:
        ```json
        {{
          "action": "create_order",
          "order_items": [
            {{
              "drink": "아메리카노",
              "size": "미디움",
              "temperature": "핫",
              "quantity": 4,
              "add_ons": [],
              "extra_shots": 0
            }}
          ]
        }}
        ```

    - **Requesting Order Summary**:
      - **Customer Input**: "내가 지금까지 뭘 주문했지?"
      - **Natural Language Response**: "지금까지 주문하신 내용은 다음과 같습니다:
      -아메리카노 4잔(핫, 미디움)
      -카페라떼 라지 2잔 (핫, 라지)"
      - **JSON Output**: None (as it is just a summary request without any new action).

    - **Modifying an Existing Order**:
      - **Customer Input**: "주문한거 아이스 라떼로 바꿔줘."
      - **Natural Language Response**: "주문이 아메리카노에서 아이스 라떼로 변경되었습니다. 주문하신 내용은 다음과 같습니다:
      -라떼 1잔 (아이스, 미디움)"
      - **JSON Output**:
        ```json
        {{
          "action": "modify_order",
          "old_drink": "아메리카노",
          "new_drink": "라떼",
          "size": "미디움",
          "temperature": "아이스",
          "quantity": 1,
          "add_ons": [],
          "extra_shots": 0
        }}
        ```

    - **Short Names or Misspellings**:
      - Recognize common shorthand or misspellings. For example:
        - "아아" should be interpreted as "아이스 아메리카노".
        - "뜨아" should be interpreted as "핫 아메리카노".

    - **Unavailable Items**:
      - If the customer requests an item not on the menu, respond politely and recommend a similar item if available.
      - **Example**:
        - **Customer Input**: "초코라떼 주세요."
        - **Natural Language Response**: "죄송합니다, 초코라떼는 메뉴에 없습니다. 대신 초콜릿라떼를 추천드립니다."
        - **JSON Output**:
          ```json
          {{
            "action": "recommend_closest_item",
            "requested_item": "초코라떼",
            "recommended_item": "초콜릿라떼"
          }}
          ```

    - **Order Confirmation**:
      - **Customer Input**: "주문 완료할게요."
      - **Natural Language Response**: "주문이 완료되었습니다. 결제는 카드리더기를 사용해주세요. 감사합니다."
      - **JSON Output**: Clear/reset for the next session.

    **Response Rules**:
    - Treat each new input as part of the same order until "주문 완료할게" is received, which finalizes the order.
    - Always confirm the latest action first in the natural language response, followed by a full order summary.
    - Ensure each JSON output reflects only the latest request, not the entire order history.

    Based on this information, generate the appropriate natural language response and JSON output
    if **Current Conversation History**:
    {formatted_history}

    and **Customer's New Input**: "{user_input}".
    """

    print("Formatted History for Current Prompt:", formatted_history)  # Debugging: Print formatted history
    return base_prompt.strip()

# Function to get response from Together API
def get_response(prompt):
    response = together_client.chat.completions.create(
        model="meta-llama/Meta-Llama-3.1-405B-Instruct-Turbo",
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": prompt}
        ],
        max_tokens=500,
        temperature=0.1,
        top_p=1,
        top_k=50,
        repetition_penalty=1,
        stop=["<|eot_id|>"]
    )
    return response.choices[0].message.content

# Function to parse response into natural language and JSON output
def parse_response(response):
    try:
        natural_language_response, json_output = response.split("JSON Output:")
        json_data = json.loads(json_output.strip())
        return natural_language_response.strip(), json_data
    except ValueError as e:
        print("Error parsing response:", e)
        return response, None

# Main interaction loop
print("Welcome to the virtual coffee kiosk! What would you like to order?")
input_counter = 1

while not order_confirmed:
    # Take user input
    user_input = input("Customer: ")

    # Check if customer confirms the order
    if "주문 완료할게" in user_input:
        order_confirmed = True
        print("Kiosk: 주문이 완료되었습니다. 결제는 카드리더기를 사용해주세요. 감사합니다.")
        conversation_history.clear()
        continue

    # Add the latest user input to the conversation history
    conversation_history.append(f"Customer's {input_counter} Input: {user_input}")
    input_counter += 1

    # Generate the prompt based on the conversation history and new user input
    prompt = generate_prompt(conversation_history, user_input)

    # Get the model's response
    response = get_response(prompt)

    # Parse the model's response
    natural_language_response, json_output = parse_response(response)

    # Print the final natural language response for the customer
    print("Kiosk:", natural_language_response)

    # Special handling for order summary requests
    if "내가 지금까지 뭘 주문했지" in user_input:
        # Provide cumulative order summary as a natural response without JSON output
        print("Kiosk:", "지금까지의 주문내역입니다:\n" + "\n".join(conversation_history))

    # Handle order cancellation requests
    if "주문 취소" in user_input:
        # Clear conversation history for a reset, or selectively based on more detailed criteria if needed
        print("Kiosk: 주문이 취소되었습니다. 새 주문을 시작해 주세요.")
        conversation_history.clear()
        input_counter = 1

# End of the session
print("Thank you for using the coffee kiosk!")

Welcome to the virtual coffee kiosk! What would you like to order?
Customer: 핫 아메리카노 2잔이랑 아이스 아메리카노 3잔 주세요
Formatted History for Current Prompt: Customer's 1 Input: 핫 아메리카노 2잔이랑 아이스 아메리카노 3잔 주세요
Error parsing response: Expecting value: line 1 column 1 (char 0)
Kiosk: Since the customer's new input is identical to their previous input, we can treat this as an addition to the existing order.

**Natural Language Response:**
핫 아메리카노 2잔과 아이스 아메리카노 3잔이 추가 주문되었습니다. 지금까지 주문하신 내용은 다음과 같습니다:
- 핫 아메리카노 4잔 (핫, 미디움)
- 아이스 아메리카노 6잔 (아이스, 미디움)

**JSON Output:**
```json
{
  "action": "add_item",
  "order_items": [
    {
      "drink": "아메리카노",
      "size": "미디움",
      "temperature": "핫",
      "quantity": 2,
      "add_ons": [],
      "extra_shots": 0
    },
    {
      "drink": "아메리카노",
      "size": "미디움",
      "temperature": "아이스",
      "quantity": 3,
      "add_ons": [],
      "extra_shots": 0
    }
  ]
}
```
Note that the JSON output only reflects the latest request, which is the addition of 

KeyboardInterrupt: Interrupted by user

In [ ]:
#never mind this  is the best
import json
from together import Together
from openai import OpenAI


# Initialize Together and OpenAI clients with your API keys
together_client = Together(api_key="8a7afb786e80870e79e094bb105c9fd6f9ace6acff04c80fcf4f71d18e1aecf4")
openai_client = OpenAI(api_key="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", base_url="https://api.perplexity.ai")

# Initialize conversation history, summary history, and order confirmation flag
conversation_history = []  # Stores each customer input as a string entry in the format: "Customer's X Input: [input]"
summary_history = []       # Stores each cumulative summary of orders
order_confirmed = False    # Tracks if the order is finalized

# Function to generate prompt based on conversation history and new input
def generate_prompt(conversation_history, user_input):
    # Format the conversation history and cumulative summary as a single string
    formatted_history = "\n".join(conversation_history) if conversation_history else "none"
    formatted_summary = "\n".join(summary_history) if summary_history else "none"

    base_prompt = f"""
    You are operating a virtual coffee kiosk that receives speech-to-text (STT) inputs from customers placing coffee orders. Your role is to understand and process these inputs, respond naturally in Korean, and generate a structured JSON file with the correct details for backend processing.

    **Key Requirements**:
    - **Menu Items**: The kiosk offers the following drinks:
        - Hot and Iced Coffee: 아메리카노, 라떼, 카푸치노, 카페모카, 바닐라라떼, 에스프레소, 카라멜마끼아또
        - Tea: 허브티, 홍차
        - Specialty Drinks: 초콜릿라떼, 레몬에이드, 복숭아아이스티, 딸기스무디, 망고스무디, 키위주스, 토마토주스
    - **Default Values**:
        - Use default size "미디움" and temperature "핫" only if the customer does not specify these details.
    - **Do Not Make Assumptions**:
        - If the customer specifies temperature or size, do not override it with defaults. For instance, if they say "아이스 라떼 두잔 주세요", the output should indicate "아이스" without changing it to "핫".

    **Customer Input and Expected Output Format**:
    - Each response should have:
      1. **Natural Language Confirmation**: Respond in Korean, starting with an action confirmation such as "[Drink] [quantity] 주문되었습니다." and follow with a full summary of all items ordered so far, beginning with "지금까지 주문하신 내용은 다음과 같습니다:".(also if there are any instance in history it should be added after this sentence)
      2. **Structured JSON Output**: Each JSON output should only contain the items directly requested in the latest input, not a full history.

    **JSON Output Format**:
    - The JSON should be structured as follows:
      ```json
      {{
          "action": "[action_type]",
          "order_items": [
              {{
                  "drink": "[Drink Name]",
                  "size": "[Size]",
                  "temperature": "[Temperature]",
                  "quantity": [Quantity],
                  "add_ons": [List of add-ons if any],
                  "extra_shots": [Number of extra shots if any]
              }}
          ]
      }}
      ```
      - **Example JSON Output**:
        ```json
        {{
            "action": "create_order",
            "order_items": [
                {{
                    "drink": "아메리카노",
                    "size": "미디움",
                    "temperature": "핫",
                    "quantity": 1,
                    "add_ons": [],
                    "extra_shots": 0
                }}
            ]
        }}
        ```

    **Available Actions for JSON Output**:
    - **create_order**: For new drink orders.
    - **add_item**: For adding a new item to the current order.
    - **modify_order**: For changing an existing item (e.g., modifying size or temperature).
    - **cancel_order**: To remove an order item or reset the order.
    - **recommend_closest_item**: If a requested item is unavailable, recommend the closest item.
    - **show_order_summary**: Display a summary of all items ordered so far.
    - **complete_order**: Finalize the order after confirmation.

    **Specific Scenarios and Expected Outputs**:
    - **Creating a New Order**:
      - **Customer Input**: "아메리카노 4잔 주세요."
      - **Natural Language Response**: "아메리카노 4잔 주문되었습니다. 지금까지 주문하신 내용은 다음과 같습니다:
      -아메리카노 4잔 (핫, 미디움)"
      - **JSON Output**:
        ```json
        {{
          "action": "create_order",
          "order_items": [
            {{
              "drink": "아메리카노",
              "size": "미디움",
              "temperature": "핫",
              "quantity": 4,
              "add_ons": [],
              "extra_shots": 0
            }}
          ]
        }}
        ```

    - **Requesting Order Summary**:
      - **Customer Input**: "내가 지금까지 뭘 주문했지?"
      - **Natural Language Response**: "지금까지 주문하신 내용은 다음과 같습니다:
      -아메리카노 4잔(핫, 미디움)
      -카페라떼 라지 2잔 (핫, 라지)"
      - **JSON Output**: None (as it is just a summary request without any new action).

    - **Modifying an Existing Order**:
      - **Customer Input**: "주문한거 아이스 라떼로 바꿔줘."
      - **Natural Language Response**: "주문이 아메리카노에서 아이스 라떼로 변경되었습니다. 주문하신 내용은 다음과 같습니다:
      -라떼 1잔 (아이스, 미디움)"
      - **JSON Output**:
        ```json
        {{
          "action": "modify_order",
          "old_drink": "아메리카노",
          "new_drink": "라떼",
          "size": "미디움",
          "temperature": "아이스",
          "quantity": 1,
          "add_ons": [],
          "extra_shots": 0
        }}
        ```

    - **Short Names or Misspellings**:
      - Recognize common shorthand or misspellings. For example:
        - "아아" should be interpreted as "아이스 아메리카노".
        - "뜨아" should be interpreted as "핫 아메리카노".

    - **Unavailable Items**:
      - If the customer requests an item not on the menu, respond politely and recommend a similar item if available.
      - **Example**:
        - **Customer Input**: "초코라떼 주세요."
        - **Natural Language Response**: "죄송합니다, 초코라떼는 메뉴에 없습니다. 대신 초콜릿라떼를 추천드립니다."
        - **JSON Output**:
          ```json
          {{
            "action": "recommend_closest_item",
            "requested_item": "초코라떼",
            "recommended_item": "초콜릿라떼"
          }}
          ```

    - **Order Confirmation**:
      - **Customer Input**: "주문 완료할게요."
      - **Natural Language Response**: "주문이 완료되었습니다. 결제는 카드리더기를 사용해주세요. 감사합니다."
      - **JSON Output**: Clear/reset for the next session.

    **Response Rules**:
    - Treat each new input as part of the same order until "주문 완료할게" is received, which finalizes the order.
    - Always confirm the latest action first in the natural language response, followed by a full order summary.
    - Ensure each JSON output reflects only the latest request, not the entire order history.

    Based on this information, generate the appropriate natural language response and JSON output
    if **Current Conversation History**:
    {formatted_history}

    **Cumulative Order Summary So Far**:
    {formatted_summary}

    and **Customer's New Input**: "{user_input}"
    """
    return base_prompt.strip()

# Function to get response from Together API
def get_response(prompt):
    response = together_client.chat.completions.create(
        model="meta-llama/Llama-3.2-11B-Vision-Instruct-Turbo",
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": prompt}
        ],
        max_tokens=500,
        temperature=0.1,
        top_p=1,
        top_k=50,
        repetition_penalty=1,
        stop=["<|eot_id|>"]
    )
    return response.choices[0].message.content

# Function to parse response into natural language and JSON output
def parse_response(response):
    try:
        natural_language_response, json_output = response.split("JSON Output:")
        json_data = json.loads(json_output.strip())
        return natural_language_response.strip(), json_data
    except ValueError as e:
        print("Error parsing response:", e)
        return response, None

# Main interaction loop
print("Welcome to the virtual coffee kiosk! What would you like to order?")
input_counter = 1

while not order_confirmed:
    # Take user input
    user_input = input("Customer: ")

    # Check if customer confirms the order
    if "주문 완료할게" in user_input:
        order_confirmed = True
        print("Kiosk: 주문이 완료되었습니다. 결제는 카드리더기를 사용해주세요. 감사합니다.")
        conversation_history.clear()
        summary_history.clear()
        continue

    # Add the latest user input to the conversation history
    conversation_history.append(f"Customer's {input_counter} Input: {user_input}")
    input_counter += 1

    # Generate the prompt based on the conversation history and new user input
    prompt = generate_prompt(conversation_history, user_input)

    # Get the model's response
    response = get_response(prompt)

    # Parse the model's response
    natural_language_response, json_output = parse_response(response)

    # Save natural language order summary to `summary_history`
    if natural_language_response:
        summary_line = natural_language_response.split("\n", 1)[1]  # Extracts summary part
        summary_history.append(summary_line)

    # Print the final natural language response for the customer
    print("Kiosk:", natural_language_response)

    # Special handling for order summary requests
    if "내가 지금까지 뭘 주문했지" in user_input:
        # Provide cumulative order summary as a natural response without JSON output
        print("Kiosk:", "지금까지의 주문내역입니다:\n" + "\n".join(summary_history))

    # Handle order cancellation requests
    if "주문 취소" in user_input:
        # Clear conversation history and summary history for a reset
        print("Kiosk: 주문이 취소되었습니다. 새 주문을 시작해 주세요.")
        conversation_history.clear()
        summary_history.clear()
        input_counter = 1

# End of the session
print("Thank you for using the coffee kiosk!")


Welcome to the virtual coffee kiosk! What would you like to order?
Error parsing response: not enough values to unpack (expected 2, got 1)
Kiosk: **Natural Language Response**: 
"아이스 라떼 두잔 주문되었습니다. 지금까지 주문하신 내용은 다음과 같습니다:
-아이스 라떼 2잔 (아이스, 미디움)"

**JSON Output**:
```json
{
  "action": "create_order",
  "order_items": [
    {
      "drink": "라떼",
      "size": "미디움",
      "temperature": "아이스",
      "quantity": 2,
      "add_ons": [],
      "extra_shots": 0
    }
  ]
}
```
In this response, the natural language response confirms the order of two servings of iced latte and provides a summary of the current order. The JSON output reflects the latest request, indicating a new order with two servings of iced latte, medium size, and iced temperature.
Error parsing response: not enough values to unpack (expected 2, got 1)
Kiosk: **Natural Language Response**: "아이스 아메리카노에 샷 추가해서 두잔 주문되었습니다. 지금까지 주문하신 내용은 다음과 같습니다:
-아이스 라떼 2잔 (아이스, 미디움)
-아이스 아메리카노 2잔 (아이스, 미디움) with 샷"

**JSON Output**:
```json